<a href="https://colab.research.google.com/github/dayody/Build_Generative_AI_APP/blob/main/GenerativeAI_Coded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code-focused Generative AI App Development with ChatGPT, LangChain and Streamlit

Here we will build a Code-focused Generative AI App Development chatbot using LangChain and Streamlit with the following features:

- Custom Landing Page
- Conversational memory
- Result streaming capabilities (Real-time output)

## Install App and LLM dependencies

In [1]:
!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11
!pip install streamlit==1.32.2
!pip install pyngrok==7.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 700.3 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: langsmith
    Found existing installatio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 527.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 396.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully unin

## Load OpenAI API Credentials

Here we load it from a file so we don't explore the credentials on the internet by mistake

**Using Colab Secrets Manager to store API credentials:**

It is recommended to use Colab's built-in Secrets Manager to securely store your API key instead of keeping it in a file.

1. Click on the "🔑" icon in the left sidebar.
2. Click on "Add new secret".
3. Enter `OPENAI_API_KEY` as the "Name" and paste your OpenAI API key in the "Value" field.
4. Make sure "Notebook access" is enabled for this secret.

Then you can access the API key using the following code:

In [5]:
from google.colab import userdata

api_creds = {
    'OPENAI_API_KEY': userdata.get('OPENAI_API_KEY')
}

# You can access the key like this:
# openai_key = api_creds['OPENAI_API_KEY']

In [7]:
import os

os.environ['OPENAI_API_KEY'] = api_creds['OPENAI_API_KEY']

## Write the app code here and store it in a py file

In [27]:
# the following line is a magic command
# that will write all the code below it to the python file app.py
# we will then deploy this app.py file on the cloud server where colab is running
# if you have your own server you can just write the code in app.py and deploy it directly
%%writefile app.py

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.callbacks.base import BaseCallbackHandler
from operator import itemgetter
import streamlit as st

# Customize initial app landing page
st.set_page_config(page_title="AI Assistant", page_icon="🤖")
st.title("Welcome! I’m the 3Cloud AI Assistant 🤖")

# Manages live updates to a Streamlit app's display by appending new text tokens
# to an existing text stream and rendering the updated text in Markdown
class StreamHandler(BaseCallbackHandler):
  def __init__(self, container, initial_text=""):
    self.container = container
    self.text = initial_text

  def on_llm_new_token(self, token: str, **kwargs) -> None:
    self.text += token
    self.container.markdown(self.text)

# Load a connection to ChatGPT LLM
chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.1,
                     streaming=True)

# Add a basic system prompt for LLM behavior
SYS_PROMPT = """
              Act as a helpful assistant and answer questions to the best of your ability.
              Do not make up answers.
              """

# Create a prompt template for langchain to use history to answer user prompts
prompt = ChatPromptTemplate.from_messages(
  [
    ("system", SYS_PROMPT),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
  ]
)

# Create a basic llm chain
llm_chain = (
  prompt
  |
  chatgpt
)

# Store conversation history in Streamlit session state
streamlit_msg_history = StreamlitChatMessageHistory()

# Create a conversation chain
conversation_chain = RunnableWithMessageHistory(
  llm_chain,
  lambda session_id: streamlit_msg_history,  # Accesses memory
  input_messages_key="input",
  history_messages_key="history",
)

# Shows the first message when app starts
if len(streamlit_msg_history.messages) == 0:
  streamlit_msg_history.add_ai_message("How can I help you?")

# Render current messages from StreamlitChatMessageHistory
for msg in streamlit_msg_history.messages:
  st.chat_message(msg.type).write(msg.content)

# If user inputs a new prompt, display it and show the response
if user_prompt := st.chat_input():
  st.chat_message("human").write(user_prompt)
  # This is where response from the LLM is shown
  with st.chat_message("ai"):
    # Initializing an empty data stream
    stream_handler = StreamHandler(st.empty())
    config = {"configurable": {"session_id": "any"},
              "callbacks": [stream_handler]}
    # Get llm response
    response = conversation_chain.invoke({"input": user_prompt},
                                         config)

Overwriting app.py


## Start the app

In [28]:
!streamlit run app.py --server.port=8989 &>./logs.txt &

In [29]:
from pyngrok import ngrok
import yaml
from google.colab import userdata

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken
# Get your authtoken from Colab secrets
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://b3d3215755e7.ngrok-free.app
